![NeuronUnit Logo](https://raw.githubusercontent.com/scidash/assets/master/logos/neuronunit-logo-text.png)
# Chapter 3
Back to [Chapter 2](chapter2.ipynb)

In the first chapter we used a dummy model and simple test to illustrate some basic concepts.  Here we'll use NeuronUnit in a more interesting way: let's validate a real ion channel model's IV (current vs voltage) curve against data from a published experiment.

In [22]:
!git clone https://github.com/openworm/ChannelWorm

Cloning into 'ChannelWorm'...
remote: Enumerating objects: 4442, done.
remote: Total 4442 (delta 0), reused 0 (delta 0), pack-reused 4442
Receiving objects: 100% (4442/4442), 43.65 MiB | 3.17 MiB/s, done.
Resolving deltas: 100% (2221/2221), done.


In [2]:
# First get ChannelWorm at http://github.com/openworm/channelworm
from pathlib import Path
import sys
CHANNELWORM_HOME = Path('ChannelWorm').resolve()
sys.path.insert(1, str(CHANNELWORM_HOME))

In [3]:
import channelworm
if not getattr(channelworm, 'setup', False):
    channelworm.django_setup() # Adds all channelworm subdirectories to the path, including ion_channel.  
    channelworm.setup = True
from channelworm.ion_channel.models import GraphData # A class to retrieve digitized data from figures.  
from neuronunit.tests.channel import IVCurvePeakTest # A test for the current-voltage relationship 
                                                     # of a neuron or ion channel.
from neuronunit.models.channel import ChannelModel # A SciUnit model class that wraps NeuroML ion channel models.  

/home/rgerkin/miniconda3/lib/python3.8/site-packages/airspeed/__init__.py:505: FutureWarning: Possible nested set at position 8
  KEYVALSEP = re.compile(r'[ \t]*:[[ \t]*(.*)$', re.S)


### Now let's instantite the ion channel `model`.  
It will be one from a calcium channel whose subunit is thought to be encoded by the EGL-19 gene in C. elegans.

In [6]:
channel_model_name = 'EGL-19.channel' # Name of a NeuroML channel model (corresponds to the name of a NeuroML file).
channel_id = 'ca_boyle' # Identifier for the channel in that NeuroML file.
channel_dir_path = CHANNELWORM_HOME / 'models' # Location of NeuroML channel files.  
channel_file_path = channel_dir_path / ('%s.nml' % channel_model_name) # Location of the current file.  
model = ChannelModel(str(channel_file_path), channel_index=0, name=channel_model_name) # Instantiate the model.  
                                                                    # channel_index=0 in most cases, unless there
                                                                    # are multiple channels of the same type
                                                                    # in the model file.  

`ChannelModel` is a NeuronUnit `model` class that wraps an ion channel model expressed using [NeuroML](http://neuroml.org) and simulates its dynamics using [jNeuroML](https://github.com/NeuroML/jNeuroML).  **You should install both.**
### As before, all of the interaction with the model is done through its capabilities.  
In this case, those capabilities are inherited in the `ChannelModel` class definition, and implemented in the methods of that class as defined in `neuronunit.models`.  

Now we get an observation of real ion channel behavior to use for our test:

In [8]:
import quantities as pq

doi = '10.1083/jcb.200203055' # A Digital Object Identifier for (in this case) a journal article.  
fig = '2B' # A reference to a figure in that article.  
sample_data = GraphData.objects.get(graph__experiment__reference__doi=doi, 
                                    graph__figure_ref_address=fig) # If that figure has been digitized by the 
                                                                   # ChannelWorm team, we load its data here.  
obs = list(zip(*sample_data.asarray())) # Reformat this to a raw observation of current density and voltage.  
observation = {'i/C':obs[1]*pq.A/pq.F, 'v':obs[0]*pq.mV} # Make a dictionary with the appropriately valued units.
cell_capacitance = 1e-13 * pq.F # Capacitance is arbitrary if IV curves are scaled.  
observation['i'] = observation['i/C']*cell_capacitance

test = IVCurvePeakTest(observation, scale=True)

SynchronousOnlyOperation: You cannot call this from an async context - use a thread or sync_to_async.

Now that we have an observation, we can instantiate our test.  
### How did we know the observation needs the form above?  
It is stated in the tests's docstring, or can be inferred from its optional validate_observation method:
```python
class IVCurveTest(sciunit.Test):
    ...
    def validate_observation(self, observation):
            assert type(observation) is dict
            for item in ['v', 'i']:
                assert item in observation
                assert type(observation[item]) in [list,tuple] \
                    or isinstance(observation[item],np.ndarray)
    ...
```

In [ ]:
test = IVCurvePeakTest(observation)

### With the test instantiated, we can now judge the model:

In [ ]:
# Judge the model output against the experimental data
score = test.judge(model)
score.summarize()
print("The score was computed according to '%s' with raw value %s and pass cutoff %s" \
    % (score.description,score.raw,test.converter.cutoff))
print('The scaling factor for the model IV curve was %.3g' % score.related_data['scale_factor'])

### A `score` object isn't just a number.###
It is also bound to data that allow us to visualize the test result and how the score was reached:

In [ ]:
%matplotlib notebook
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams.update({'font.size':14, 'lines.linewidth':3})
score.plot()
plt.tight_layout()

That's a failing score, because those curves don't look too similar.  What was the criterion for passing?

In [ ]:
score.describe()
print("Raw score is %s" % score.raw)
score.test.converter.description

### Let's change the threshold for a passing score:

In [ ]:
score.test.converter.cutoff = pq.Quantity(10.0,'pA**2') # Make it easier to pass.  
score = test.converter.convert(score)
score.summarize() # Now shows that the test has passed.  

Now the score is changed to reflect the new cutoff.  This cutoff is up the person doing the testing!  

### On to [Chapter 4](chapter4.ipynb)!